In [1]:
import os, sys, time, resource, re, gc, shutil
import operator
from multiprocess import Pool
from functools import partial
from urllib.parse import urlparse, parse_qsl
from django.db.models import Count
import matplotlib
matplotlib.use('pgf')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import django
from adjustText import adjust_text

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *
from django.db.models import F, Sum, Count
from matplotlib import gridspec

from utils.utils import *


import statsmodels.formula.api as sm, statsmodels.stats.api as sms

pgf_with_latex = {
    "text.usetex": True,            # use LaTeX to write all text
    "pgf.rcfonts": False,           # Ignore Matplotlibrc
    "text.latex.unicode": True,
    "pgf.preamble": [
        #r"\usepackage[utf8x]{inputenc}",
        r"\usepackage{xcolor}"
    ],
    "pgf.texsystem" : "xelatex",
    "figure.figsize": [7.2,5]
}
matplotlib.rcParams.update(pgf_with_latex)


from run_id import run_id

run_id = 662

runstat = RunStats.objects.get(pk=run_id)
topics = DynamicTopic.objects.filter(run_id=run_id)

/usr/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  return f(*args, **kwds)


In [2]:
tsums = topics.aggregate(
    ip_score=Sum('ipcc_score'),
    score=Sum('ipcc_time_score')
)
tsums

topics.update(
    ipcc_share=F('ipcc_score')/tsums['ip_score'],
    share=F('ipcc_time_score')/tsums['score']
)

100

In [3]:
def calculate_deviations(df):
    df['deviation'] = df['ipcc_share'] - df['share']
    df['abs_md'] = abs(df['deviation'])

    md = df['deviation'].max()
    rae = df['abs_md'].mean()
    lh = df['abs_md'].sum() / 2

    df['representation'] = df['ipcc_share'] / df['share']  
    
    df_disp = {'MD':md,'Rae':rae,'L-H':lh}
    return [df,df_disp]


In [4]:



df = pd.DataFrame.from_dict(
    list(topics.values(
        'title',
        'score',
        'ipcc_coverage',
        'share',
        'ipcc_score',
        'ipcc_share',
        'ipcc_time_score',
        'primary_wg'
    ))
)

df, df_disp  = calculate_deviations(df)


df.sort_values('representation').head()

,ipcc_coverage,ipcc_score,ipcc_share,ipcc_time_score,primary_wg,score,share,title,deviation,abs_md,representation
39,0.029822,6.996709,0.000926,234.618406,3,668.346077,0.004727,"{adsorpt, adsorb, capac}",-0.003801,0.003801,0.195844
72,0.039887,10.607514,0.001404,265.938777,2,385.865575,0.005358,"{root, respir, fine}",-0.003955,0.003955,0.261946
13,0.040632,24.862808,0.003290,611.903042,1,1298.677539,0.012328,"{process, captur, reaction}",-0.009039,0.009039,0.266838
38,0.041303,10.294637,0.001362,249.246622,3,492.612029,0.005022,"{wast, landfil, compost}",-0.003660,0.003660,0.271245
57,0.044705,20.371524,0.002695,455.684960,2,683.251724,0.009181,"{sediment, deposit, eros}",-0.006486,0.006486,0.293588


In [5]:

from matplotlib import ticker


plt.rcParams["figure.figsize"] = [7.2,5]

def plot_representation(df, ax, nts=5, xspace=3,yspace = 0.15, fmin=None, fmax=None):
    
    md = df['deviation'].max()
    rae = df['abs_md'].mean()
    lh = df['abs_md'].sum() / 2

    pdf = df.sort_values('representation',ascending=False).reset_index()#.set_index('title')

    pdf = pdf[pdf['share'] > 0.001]

    pdf['lrep'] = np.log(pdf['representation'])
    #pdf['lrep'] = pdf['representation']
    pdf.set_index('title')['lrep'].plot(kind="bar",ax=ax,color="grey")

    i = 0

    rmax = pdf['lrep'].max()
    if fmax:
        rmax = fmax
    rmin = pdf['lrep'].min()
    if fmin:
        rmin = -fmin
    
    for index, row in pdf.head(nts).iterrows():
        i+=1
        s = round(row['ipcc_share']*100,1)
        v = round(row['share']*100,1)
        ax.annotate(
            s="{} [{},{}]".format(row['title'],s,v),
            xy=(-0.5+i,row['lrep']),
            xytext=((xspace-3)*-1+i*xspace,rmax-rmax*yspace*(i-1)),
            arrowprops=dict(
                facecolor='black', 
                #shrink=0.05,
                width=0.1,
                headwidth=0.2
            ),
            ha="left",
            va="bottom"
        )

    i = 0
    for index, row in pdf.sort_values('representation').head(nts).iterrows():
        i+=1
        s = round(row['ipcc_share']*100,1)
        v = round(row['share']*100,1)
        ax.annotate(
            s="{} [{},{}]".format(row['title'],s,v),
            xy=(len(pdf)-0.5-i,row['lrep']),
            xytext=(len(pdf)+xspace-3-i*xspace,rmin-rmin*yspace*(i-1)),
            arrowprops=dict(
                facecolor='black', 
                #shrink=0.05,
                width=0.1,
                headwidth=0.2
            ),
            ha="right",
            va="top"
        )
        
    #ax.set_yscale('log')
    
    
    if rmin > -1:
        rmin = -1
    if np.exp(rmax) < 2:
        rmax = np.log(2.1)
    
    lmin = int(1/np.exp(rmin)//1)*-1
    lmax = int(np.exp(rmax)//1)
    


    ytick_labels = [i for i in range(lmin,lmax+1) if i not in [-1,1]]
    yticks = []
    for x in ytick_labels:
        if x < 0:
            y = np.log(-1/x)
        elif x==0:
            y = 0
        else:
            y = np.log(x)
        yticks.append(y)
    
    ax.set_yticks(yticks)
    ax.set_yticklabels(ytick_labels)
    
    #ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%.1f"))
        
    rmax = pdf['lrep'].max()
    rmin = pdf['lrep'].min()
    
    ax.text(
        len(pdf)*0.066,-0.2,
        "MD: {:.3f}\nRae: {:.3f}\nL-H: {:.3f}".format(md,rae,lh),
        va="top",
        ha="left",
        bbox={'facecolor':'red', 'alpha':0.3,'pad':10}
   )

    #ax.text(1,-1*0.2,"MD: {:.3f}".format(md))   
    #ax.text(1,-1*0.4,"Rae: {:.3f}".format(rae))   
    #ax.text(1,-1*0.6,"L-H: {:.3f}".format(lh))   
    
    ax.get_xaxis().set_visible(False)#.set_ticks([])
    
    ax.set_ylim((rmin+rmin*0.1,rmax+rmax*0.15))

fig, ax = plt.subplots()
plot_representation(df,ax,nts=10,xspace=6,yspace=0.07)
plt.tight_layout()
plt.savefig('../plots/ipcc_representation/ipcc_rep_{}_all.pdf'.format(run_id),bbox_inches='tight')
plt.show()

In [6]:
rmin = -1.5
rmax = 1
lmin = int(1/np.exp(rmin)//1)*-1
lmax = int(np.exp(rmax)//1)

ytick_labels = [i for i in range(lmin,lmax) if i not in [-1,1]]
print(ytick_labels)
yticks = []
for x in ytick_labels:
    if x < 0:
        x = np.log(-1/x)
    elif x==0:
        x = 0
    else:
        x = np.log(x)
    yticks.append(x)
#yticks = [np.log(x) if x!=0 else 0 for x in ytick_labels ]
print(yticks)

[-4, -3, -2, 0]
[-1.3862943611198906, -1.0986122886681098, -0.6931471805599453, 0]


plt.rcParams["figure.figsize"] = [16,12]

stat = RunStats.objects.get(pk=run_id)

fig = plt.figure()


disp = pd.DataFrame(columns=['AR','MD','Rae','L-H'])
#fig, axs = plt.subplots(2,3,sharey=True)

for i, tp in enumerate(stat.periods.filter(n__lt=6)):
    if i == 0:
        ax = fig.add_subplot(2,3,i+1)
        ax1 = ax
    else:
        ax = fig.add_subplot(2,3,i+1,sharey=ax1)
    print(tp)
    tdts = TimeDTopic.objects.filter(dtopic__run_id=run_id,period=tp)
    
    df = pd.DataFrame.from_dict(
        list(tdts.values(
            'dtopic__title',
            'score',
            'share',
            'ipcc_score',
            'ipcc_share',
            #'ipcc_time_score'
        ))
    )
    
    df = df.rename(columns={'dtopic__title':'title'})
    
    df, df_disp  = calculate_deviations(df)
    df_disp['AR'] = tp.title   
    disp = disp.append(pd.DataFrame.from_dict([df_disp]))

    plot_representation(df, ax, 2.3,1)
    
    ax.set_title(tp.title)
    
    if tp.title=="AR5":
        ax = fig.add_subplot(2,3,6)
        
        disp = disp.set_index('AR')
        
        disp.plot(ax=ax)
        
        ax.set_title("Disproportionality")
    
    
    
    #plt.savefig('../plots/ipcc_representation/ipcc_rep_{}_{}.png'.format(run_id,tp.title),bbox_inches='tight')
    #plt.show()

plt.savefig('../plots/ipcc_representation/ipcc_rep_{}_ARS.png'.format(run_id),bbox_inches='tight')
plt.show()

disp.head()

In [7]:
tds = topics.filter(timedtopic__period__n__lt=6).values(
    'title','timedtopic__period__title','timedtopic__period__n','timedtopic__score','score'
).order_by('id','timedtopic__period__n')

tdf = pd.DataFrame.from_dict(list(tds))

#tdf['ys'] = tdf[]

tdf['share'] = tdf['timedtopic__score'] / tdf['score']

tdf['ys'] = tdf['timedtopic__period__n'] * tdf['share']


tdf.head(12)
#tdf.groupby('')

,score,timedtopic__period__n,timedtopic__period__title,timedtopic__score,title,share,ys
0,549.140042,1,AR1,0.000000,"{industri, sector, intens}",0.000000,0.000000
1,549.140042,2,AR2,7.875215,"{industri, sector, intens}",0.014341,0.028682
2,549.140042,3,AR3,19.710819,"{industri, sector, intens}",0.035894,0.107682
3,549.140042,4,AR4,45.982418,"{industri, sector, intens}",0.083735,0.334941
4,549.140042,5,AR5,131.261191,"{industri, sector, intens}",0.239030,1.195152
5,849.147223,1,AR1,1.544717,"{surfac, layer, atmospher}",0.001819,0.001819
6,849.147223,2,AR2,25.276148,"{surfac, layer, atmospher}",0.029767,0.059533
7,849.147223,3,AR3,70.898813,"{surfac, layer, atmospher}",0.083494,0.250482
8,849.147223,4,AR4,150.006147,"{surfac, layer, atmospher}",0.176655,0.706620
9,849.147223,5,AR5,306.147135,"{surfac, layer, atmospher}",0.360535,1.802674


In [8]:
means = tdf.groupby('title')['ys'].mean()

means = pd.DataFrame({'ys' : tdf.groupby('title')['ys'].mean()}).reset_index()

mdf = df.merge(means)

mdf.head()

means.sort_values('ys',ascending=False).head()



,title,ys
78,"{site, differ, locat}",0.603189
79,"{snow, cover, winter}",0.595524
55,"{ozon, stratospher, tropospher}",0.591454
66,"{record, dure, holocen}",0.589496
20,"{delta, isotop, thousand}",0.575908


In [9]:
for name, group in tdf.groupby('title'):
    l = []
    for index, y in group.iterrows():
        print(y.share)
        l = l + [y.timedtopic__period__n] * round(y.share*100)
        
    print(l)
    
    print(np.mean(l))
    
    mdf.loc[mdf['title']==name]['av_year'] = np.mean(l)
    
    break

0.000703195695092464
0.015701485986982407
0.038172013285445956
0.10987530112466837
0.4379904856021453
[2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
4.590163934426229


/home/galm/software/django/tmvenv/lib/python3.4/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [10]:
def year_av(x):
    group = tdf[tdf['title']==x['title']]
    l = []
    for index, y in group.iterrows():
        l = l + [y.timedtopic__period__n] * round(y.share*100)
    return np.mean(l)

mdf['year_av'] = df.apply(year_av,axis=1)

mdf.head()

,ipcc_coverage,ipcc_score,ipcc_share,ipcc_time_score,primary_wg,score,share,title,deviation,abs_md,representation,ys,year_av
0,0.190183,73.651337,0.009745,387.266284,2,643.056244,0.007803,"{uncertainti, estim, paramet}",0.001943,0.001943,1.248966,0.520801,4.625000
1,0.098559,46.522562,0.006156,472.027127,1,762.489285,0.009510,"{activ, human, enzym}",-0.003355,0.003355,0.647256,0.555215,4.590164
2,0.085966,49.460300,0.006544,575.349182,2,970.789572,0.011592,"{elev, ambient, leaf}",-0.005048,0.005048,0.564553,0.487484,4.172414
3,0.199760,180.228730,0.023847,902.228343,2,1371.619267,0.018178,"{increas, decreas, effect}",0.005669,0.005669,1.311859,0.528593,4.306452
4,0.119510,113.316197,0.014993,948.172778,1,1535.651765,0.019103,"{data, estim, use}",-0.004110,0.004110,0.784845,0.523223,4.366667


In [11]:
x = 'year_av'
y = 'representation'

fig, ax = plt.subplots()

cmap = {1: "#66c2a5", 2: "#fc8d62" , 3: "#8da0cb"}
colors = [cmap[i] for i in mdf['primary_wg']]


mdf.plot.scatter(
    x,y,s=mdf['score']*0.03,
    c = colors,ax=ax
)

z = np.polyfit(x=mdf[x], y=mdf[y], deg=1)
p = np.poly1d(z)
mdf['trendline'] = p(mdf.loc[:, x])

plt.plot(mdf[x],mdf['trendline'])

qs = 5

mdf['x_q'] = pd.qcut(mdf[x], qs, labels=False)
mdf['y_q'] = pd.qcut(mdf[y], qs, labels=False)
texts = []


#for i, x in mdf[mdf['year_av']> 5.42].iterrows():
for i, row in mdf[(mdf['x_q'].isin([0,qs-1])) & (mdf['y_q'].isin([0,qs-1]))].iterrows():
    texts.append(plt.text(row[x],row[y],row['title'],ha='center', va='center'))
    
adjust_text(texts, 
            arrowprops=dict(arrowstyle="-", color='black', lw=0.5))

plt.axhline(1,c="grey")
plt.axvline(mdf[x].median(),c="grey")



# Put labels on the quadrants
# x
d1 = mdf[x].median()-plt.xlim()[0]
d2 = mdf[x].median()-plt.xlim()[1]

xlabpoints = [mdf[x].median()-d1/2,mdf[x].median()-d2/2]
xrange = mdf[x].max()-mdf[x].min()
xlabpoints = [mdf[x].min()+xrange*0.15,mdf[x].min()+xrange*0.85]
labels = [
    ['Under-represented,\nolder topics','Under-represented,\nnewer topics'],
    ['Over-represented,\nolder topics','Over-represented,\nnewer topics']
]

pylims = plt.ylim()

tpad = (pylims[1]-pylims[0])*0.13


ops = [operator.lt,operator.gt]


for i in [0,1]:
    for j in [0,1]:
        if j==0:
            tpadx=tpad*-1
        else:
            tpadx=tpad*0.75
            
        q = mdf[(ops[i](mdf[x],mdf[x].median())) & (ops[j](mdf[y],1))]
        qshare = q['score'].sum()/mdf['score'].sum()
        plt.text(
            xlabpoints[i],
            pylims[j]+tpadx,
            labels[j][i] + " {:.0%}".format(qshare),
            va="center",ha="center",
            bbox={'facecolor':'red', 'alpha':qshare*2-0.25, 'pad':10},
            fontsize=7
        )

        

#ax.get_yaxis().set_visible(False)
#ax.tick_params(axis=u'both', which=u'both',length=0)
#ax2.tick_params(axis=u'both', which=u'both',length=0)
plt.xlabel('Assessment period\noccurence')
plt.ylabel('Representation')

plt.tight_layout(h_pad=18)

fig.patch.set_facecolor('#f0f0f0')    
#plt.tight_layout()

plt.savefig(
    '../plots/ipcc_representation/ipcc_rep_new{}_all.pdf'.format(run_id),
    bbox_inches='tight',
    facecolor=fig.get_facecolor(),
    pad_inches=0.2
)

plt.show()

In [12]:
plt.close()

In [13]:
mdf.sort_values('x_q').head(15)

,ipcc_coverage,ipcc_score,ipcc_share,ipcc_time_score,primary_wg,score,share,title,deviation,abs_md,representation,ys,year_av,trendline,x_q,y_q
87,0.102655,43.413724,0.005744,422.907683,2,715.382857,0.008521,"{growth, econom, radial}",-0.002776,0.002776,0.674157,0.500829,4.327586,1.020122,0,1
24,0.208224,72.409217,0.009581,347.746061,1,482.870101,0.007006,"{solar, radiat, irradi}",0.002574,0.002574,1.367449,0.564247,4.242424,1.052469,0,4
55,0.100095,50.412534,0.006670,503.646136,1,739.800246,0.010147,"{flux, measur, net}",-0.003477,0.003477,0.657344,0.567363,4.292308,1.033522,0,1
71,0.068267,22.749119,0.003010,333.236565,2,507.606884,0.006714,"{litter, decomposit, organ}",-0.003704,0.003704,0.448323,0.548878,4.269841,1.042055,0,0
51,0.199364,118.228763,0.015643,593.029935,1,849.147223,0.011948,"{surfac, layer, atmospher}",0.003695,0.003695,1.309260,0.564226,4.338462,1.015991,0,3
57,0.044705,20.371524,0.002695,455.684960,2,683.251724,0.009181,"{sediment, deposit, eros}",-0.006486,0.006486,0.293588,0.563887,4.287879,1.035204,0,0
78,0.112082,59.246426,0.007839,528.600520,2,831.072346,0.010650,"{rate, accumul, respir}",-0.002811,0.002811,0.736061,0.533012,4.322581,1.022023,0,1
75,0.108076,121.853661,0.016123,1127.478165,1,1592.146233,0.022716,"{record, dure, holocen}",-0.006593,0.006593,0.709757,0.589496,4.289855,1.034453,0,1
63,0.300512,103.849334,0.013741,345.574858,1,438.025787,0.006963,"{ozon, stratospher, tropospher}",0.006778,0.006778,1.973518,0.591454,4.214286,1.063156,0,4
67,0.247476,182.699088,0.024174,738.248293,1,1017.760942,0.014874,"{atlant, sst, pacif}",0.009300,0.009300,1.625225,0.571405,4.268657,1.042505,0,4


In [14]:
mdf[mdf[y].isna()]

,ipcc_coverage,ipcc_score,ipcc_share,ipcc_time_score,primary_wg,score,share,title,deviation,abs_md,representation,ys,year_av,trendline,x_q,y_q
